##script limpio, no c si lo alcance a convertir a lo que me propuso ale :c me peldonan?

In [ ]:
##dbscan


values= np.unique(dbscan_clusters)
#print(values)
#aqui hago un intento de juntar todo (de acuerdo a lo que se tenia la ultima vez, creo ahora no es necesario)
data_dbscan = features_df.copy()  #copiamos el data de features
data_dbscan['cluster_dbs'] = dbscan_clusters #agregamos columna cluster (numero) no se si esto esta bien asi tal cual
data_dbscan = data_dbscan[['region', 'pais', 'cluster_dbs']] #achicamos y nos quedamos con eso
data_dbscan_comp = pd.merge(data_dbscan, dset_extra, how='inner') #agregamos los atributos (gini, gdp ...)
data_dbscan_comp = dset_base.merge(data_dbscan_comp.set_index('pais'), on='pais') # aqui juntamos con la data de los dias y eso

#data_dbscan_comp.head()

datas={}
for i in range(0, len(values)): #metemos lo que hay para cada cluster a un dset aparte con llave numero y lo metemos al dicct 
    llave=values[i]  #llave es el numero de cluster
    datai = data_dbscan_comp.loc[data_dbscan_comp['cluster_dbs']==values[i]] #filtramos por numero de grupo
    datas[llave]=datai  #pos se guarda


interes=[ 'gdp_rank', 'gdp_usd', 'gdp_en_salud', 'edad_media', 'gini', 'poblacion', 'taza_muerte1000' ] #col de interes
#ultra gnral y basico: valor de la media y la desviacion
for i in range(0,len(values)):
    print('info gnral cluster ', values[i], ':')
    for a in range(0,len(interes)-1):
        datos=datas[values[i]]
        print(interes[a], 'promedio: ', datos[interes[a]].mean(), ';  desviacion:   ', datos[interes[a]].std() )
        print('---------------------------------------')
    print('*********************************************************')

#boxplots
for col in interes: #iteramos sobre la lista de columnas
    par=col
    data_kmeans_comp.groupby('cluster_dbs') 
    f, ax = plt.subplots(figsize=(7,5))
    img=sns.boxplot(y=par, x= 'cluster_kmeans', data=data_kmeans_comp,  width=0.3, showfliers = True) #colorres
#los boxplot quedan bastante feos (quiza por los datos antiguos) asi que le puse densidad para ver si salia algo mas decente

#densidades    
colores= ['darkorange', 'springgreen', 'lightseagreen', 'orchid', 'chocolate', 'lightsteelblue', 'indianred', 'cadetblue', 'gold']
for col in interes:  #los colores sirven para saber el cluster
    par=col
    i=-1
    plt.figure()
    plt.title(col)
    for w in range(0,len(datas)):
        datos=datas[i]
        datos=list(datos[col])
        sns.distplot(datos, hist=False, color=colores[i])
        i+=1
    plt.close    

#paises
for i in range(0,len(values)):
    print('cluster ', values[i])
    data=datas[values[i]]
    column_valxs = data[["pais"]].values
    unique_values =  np.unique(column_valxs)    ##paises en el cluster
    print(unique_values)
    print('-----------------------------------------------------------------------------')

#matrices corr
for i in range(0,len(values)):
    print('para cluster:  ', values[i])
    data=datas[values[i]]
    mm_scaler = preprocessing.MinMaxScaler()
    data = data.drop(["fecha_observacion"  ,"region_x", "region_y","cluster_dbs", "pais", "num_dia_desde_22_enero", "num_dia_desde_primer_caso" ],axis=1)
    scaled_at = mm_scaler.fit_transform(data)
    scaled = pd.DataFrame(scaled_at)
    cormatrix = scaled.corr()
    f, ax = plt.subplots()
    sns.heatmap(cormatrix,vmax=1,vmin=-1,center=0, cmap='gnuplot2' )

#aqui bote columnas pero creo bastaba con seleccionar las de interes...



In [ ]:
##K means

#kmeans_clusters

values= np.unique(kmeans_clusters) #sacamos los distintos numeros de cluster
#print(values)
#repito como en bloque anterior cambiando lo que sea necesario
data_kmeans = features_df.copy()
data_kmeans['cluster_kmeans'] = kmeans_clusters 
data_kmeans = data_kmeans[['region', 'pais', 'cluster_kmeans']]
data_kmeans_comp = pd.merge(data_kmeans, dset_extra, how='inner')
data_kmeans_comp = dset_base.merge(data_kmeans_comp.set_index('pais'), on='pais')

#data_kmeans_comp.head()

datas={}
for i in range(0, len(values)): #metemos lo que hay para cada cluster a un dset y lo metemos en dicc datas
    llave=values[i]
    datai = data_kmeans_comp.loc[data_kmeans_comp['cluster_kmeans']==values[i]]
    datas[llave]=datai
    
interes=[ 'gdp_rank', 'gdp_usd', 'gdp_en_salud', 'edad_media', 'gini', 'poblacion', 'taza_muerte1000' ]
#gnral, mean y desv
for i in range(0,len(values)):
    print('info gnral cluster ', values[i], ':')
    for a in range(0,len(interes)):
        datos=datas[values[i]]
        print(interes[a], 'promedio: ', datos[interes[a]].mean(), ';  desviacion:   ', datos[interes[a]].std() )
        print('---------------------------------------')
    print('*******************', 'fin cluster <', values[i] ,'>************************')

    
#boxplots
for col in interes:
    par=col
    data_kmeans_comp.groupby('cluster_kmeans')
    f, ax = plt.subplots(figsize=(7,5))
    img=sns.boxplot(y=par, x= 'cluster_kmeans', data=data_kmeans_comp , width=0.3, showfliers = True) #showfliers en false no graf los outliers 

#density plots porque los boxplot no estan muy buenos de momento
colores= ['darkorange', 'springgreen', 'lightseagreen', 'orchid', 'chocolate', 'lightsteelblue', 'indianred', 'cadetblue', 'gold']

for col in interes:
    par=col
    plt.figure()
    plt.title(col)
    for i in range(0,len(datas)):
        datos=datas[i]
        datos=list(datos[col])
        sns.distplot(datos, hist=False, color=colores[i])
    plt.legend(['0','1','2'])
    plt.close


##lista paises    
for i in range(0,len(values)):
    print('cluster', values[i])
    data=datas[values[i]]
    column_valxs = data[["pais"]].values
    unique_values =  np.unique(column_valxs)    ##pais, region o pais, region ?
    print(unique_values)
    print('-----------------------------------------------------------------------------')

#matrices corr    
for i in range(0,len(values)):
    print('para cluster:  ', values[i])
    data=datas[values[i]]
    mm_scaler = preprocessing.MinMaxScaler()
    data = data.drop(["fecha_observacion"  ,"region_x", "region_y","cluster_kmeans", "pais", "num_dia_desde_22_enero", "num_dia_desde_primer_caso" ],axis=1)
    scaled_at = mm_scaler.fit_transform(data)
    scaled = pd.DataFrame(scaled_at)
    cormatrix = scaled.corr()
    f, ax = plt.subplots()
    sns.heatmap(cormatrix,vmax=1,vmin=-1,center=0, cmap='Pastel2' )


In [ ]:
###agglomerative_clusters

values= np.unique(agglomerative_clusters)
#print(values)

data_agl = features_df.copy()
data_agl['cluster_agg'] = agglomerative_clusters ####
data_agl = data_agl[['region', 'pais', 'cluster_agg']]
data_agl_comp = pd.merge(data_agl, dset_extra, how='inner')
data_agl_comp = dset_base.merge(data_agl_comp.set_index('pais'), on='pais')

#data_agl_comp.head()

datas={}
for i in range(0, len(values)): #separamos y subimos al diccionario
    llave=values[i]
    datai = data_agl_comp.loc[data_agl_comp['cluster_agg']==values[i]]
    datas[llave]=datai

interes=[ 'gdp_rank', 'gdp_usd', 'gdp_en_salud', 'edad_media', 'gini', 'poblacion', 'taza_muerte1000' ]

#basico
for i in range(0,len(values)):
    print('info gnral cluster ', values[i], ':')
    for a in range(0,len(interes)):
        datos=datas[values[i]]
        print(interes[a], 'promedio: ', datos[interes[a]].mean(), ';  desviacion:   ', datos[interes[a]].std() )
        print('---------------------------------------')
    print('*******************', 'fin cluster <', values[i] ,'>************************')

#lista paises
for i in range(0,len(values)):
    print('cluster', values[i])
    data=datas[values[i]]
    column_valxs = data[["pais"]].values
    unique_values =  np.unique(column_valxs)    
    print(unique_values)
    print('-----------------------------------------------------------------------------')

#boxplots    
for col in interes:
    par=col
    data_agl_comp.groupby('cluster_agg')
    f, ax = plt.subplots(figsize=(7,5))
    img=sns.boxplot(y=par, x= 'cluster_agg', data=data_agl_comp, width=0.3, showfliers = True)


#density plots
colores= ['darkorange', 'springgreen', 'lightseagreen', 'orchid', 'chocolate', 'lightsteelblue', 'indianred', 'cadetblue', 'gold']

for col in interes:
    par=col
    plt.figure()
    plt.title(col)
    for i in range(0,len(datas)):
        datos=datas[i]
        datos=list(datos[col])
        sns.distplot(datos, hist=False, color=colores[i])
    plt.close

#corr matrix    
for i in range(0,len(values)):
    print('para cluster:  ', values[i])
    data=datas[values[i]]
    mm_scaler = preprocessing.MinMaxScaler()
    data = data.drop(["fecha_observacion"  ,"region_x", "region_y","cluster_agg", "pais", "num_dia_desde_22_enero", "num_dia_desde_primer_caso" ],axis=1)
    scaled_at = mm_scaler.fit_transform(data)
    scaled = pd.DataFrame(scaled_at)
    cormatrix = scaled.corr()
    f, ax = plt.subplots()
    sns.heatmap(cormatrix,vmax=1,vmin=-1,center=0, cmap='rainbow' )